In [1]:
# libraries
from keras.layers import Input, Dropout, Conv2D, Dense, Flatten, Reshape
from keras.models import Model, Sequential

models = {}

Using TensorFlow backend.


In [11]:
def dense_model(input_shape, name='dense_model', **kwargs):
    # get hyper-parameters
    dropout = kwargs.get('dropout', 0.4)
    dense_units = kwargs.get('dense_units', 64)
    hidden_activation = kwargs.get('hidden_activation', 'relu')
    layers_count = max(kwargs.get('layers_count', 2), 1)
    
    # hidden network
    hidden = Sequential()
    ### first layer
    hidden.add(Dense(dense_units, 
                     activation=hidden_activation, 
                     input_shape=input_shape))
    hidden.add(Dropout(rate=dropout))
    ### append others
    for _ in range(layers_count - 1):
        hidden.add(Dense(dense_units, 
                         activation=hidden_activation))
        hidden.add(Dropout(rate=dropout))
    ### final layers
    hidden.add(Flatten())
    
    # input and output
    tf_input = Input(shape=input_shape)
    hidden_output = hidden(tf_input)
    tf_output = [Dense(9, activation='softmax')(hidden_output) for _ in range(81)]
    
    # create model
    model = Model(tf_input, tf_output)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

models['dense_model'] = dense_model

In [16]:
def conv_model(input_shape, name='conv_model', **kwargs):
    # get hyper-parameters
    dropout = kwargs.get('dropout', 0.4)
    conv_filters = kwargs.get('conv_filters', 64)
    kernel_size = kwargs.get('kernel_size', 3)
    hidden_activation = kwargs.get('hidden_activation', 'relu')
    layers_count = max(kwargs.get('layers_count', 2), 1)
    
    # hidden network
    hidden = Sequential()
    ### first layer
    hidden.add(Conv2D(filters=conv_filters,
                      kernel_size=kernel_size,
                      activation=hidden_activation,
                      padding='same',
                      input_shape=input_shape))
    hidden.add(Dropout(rate=dropout))
    ### append others
    for _ in range(layers_count - 1):
        hidden.add(Conv2D(filters=conv_filters,
                          kernel_size=kernel_size,
                          activation=hidden_activation,
                          padding='same'))
        hidden.add(Dropout(rate=dropout))
    ### final layers
    hidden.add(Flatten())
    
    # input and output
    tf_input = Input(shape=input_shape)
    hidden_output = hidden(tf_input)
    tf_output = [Dense(9, activation='softmax')(hidden_output) for _ in range(81)]
    
    # create model
    model = Model(tf_input, tf_output)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

models['conv_model'] = conv_model

In [4]:
def get_model(input_shape, model_id, model_name="sudoku_net", **kwargs):
    return models[model_id](input_shape, model_name, **kwargs)